In [8]:
import mpmath as mp
import sympy as sp
sp.init_printing()

Define our parameters
---------------------

In [73]:
precision = 50

A = sp.Integer(12)
i = 43

# A-1 / A+1 will have type sp.Rational. 
# Here we take a Float to given precision
α = ((A-1)/(A+1)).evalf(precision)**2
X = sp.log(1/α)
ξ = 1 + (α/(1-α))*sp.log(α)
u = sp.log(2E6/1)

In [74]:
%whos

Variable                         Type        Data/Info
------------------------------------------------------
A                                Integer     12
N                                function    <function N at 0x7f89251ca400>
R                                function    <function R at 0x7f89251ca620>
X                                Float       0.33410816932633238398308<...>2720037860996712218164558
expr                             Sum         Sum(Piecewise((1 + (-1 + <...> 1)/α, True)), (k, 0, i))
f                                function    <function f at 0x7f89223d0598>
i                                int         43
mp                               module      <module 'mpmath' from '/h<...>ages/mpmath/__init__.py'>
n_average_collisions             function    <function n_average_collisions at 0x7f892243bb70>
n_average_collisions_numerical   function    <function n_average_colli<...>erical at 0x7f89251ca598>
precision                        int         50
sp                 

Define symbolic solution
------------------------

In [84]:
def n_average_collisions(α, X, u, i):    
    k, m = sp.symbols("k, m")
    R_summand = sp.Sum(
        # Summand
        (-1)**m * sp.factorial(k)/sp.factorial(m) * α**m*(1-α)**(k-m+1) * (u-k*X)**m,
        # Limits
        (m, 0, k-1)
    )

    R = sp.Piecewise(
        # R(k>0):
        ((R_summand + (-1)**k*α**k*(u-k*X)**k) / (sp.factorial(k)*α**k*(1-α)**k), k > 0), 
        # R(k<=0):
        (1, k <= 0)
    )
    
    N = sp.Sum(
        # Summand
        1 + 1/α * (R*sp.exp((α*u-k*X)/(1-α)) - 1),
        # Limits
        (k, 0, i)
    )
    
    return N

Print analytical result
-----------------------

In [85]:
symbols = sp.symbols("α,X,u,i")
expr = n_average_collisions(*symbols)
expr

      i                                                                       
_____________                                                                 
╲                                                                             
 ╲            ⎧                        ⎛                       k - 1          
  ╲           ⎪                        ⎜                        ____          
   ╲          ⎪                        ⎜                        ╲             
    ╲         ⎪                        ⎜                         ╲       m  m 
     ╲        ⎪          -k         -k ⎜    k  k           k      ╲  (-1) ⋅α ⋅
      ╲       ⎪         α  ⋅(-α + 1)  ⋅⎜(-1) ⋅α ⋅(-X⋅k + u)  +    ╱  ─────────
       ╲      ⎪                        ⎜                         ╱            
        ╲     ⎪                        ⎜                        ╱             
         ╲    ⎪                        ⎜                        ‾‾‾‾          
          ╲   ⎪                        ⎝            

Evaluate result using mpmath
----------------------------
Passing in `n` implicitly uses mpmath, but we need to specify mpmath types for the constants

In [86]:
expr.evalf(n=precision, subs={symbols[0]: α,
                              symbols[1]: X,
                              symbols[2]: u,
                              symbols[3]: i})

---

Alternatively, using a direct approach
--------------------------------------

In [87]:
import mpmath as mp

def n_average_collisions_numerical(α, X, u, i):    
    def term(k):
        return 1 + 1/α * (R(α, X, u, k)*mp.exp((α*u-k*X)/(1-α)) - 1)
    return mp.nsum(term, [0, i])

def R(α, X, u, k):
    if not k:
        return 1.0
    
    def term(m):
        return (-1)**m * mp.factorial(k)/mp.factorial(m) * α**m*(1-α)**(k-m+1) * (u-k*X)**m 
              
    return (mp.nsum(term, [0, k-1]) + (-1)**k*α**k*(u-k*X)**k) / (mp.factorial(k)*α**k*(1-α)**k)

Evaluate result, using mpmath types 
-----------------------------------
These types implement operator overloading, so behave numerically

In [88]:
n_average_collisions_numerical(α, X, u, 43)

mpf('92.64722661238204283037055304094536816364614667887827596022549319642310952268529771740005055941517487691')